# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [42]:
# your code here
from sqlalchemy import create_engine
import pandas as pd
import pymysql

driver   = 'mysql+pymysql:'
user     = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
ip       = '34.65.10.136'
database = 'retail_sales'
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)
conn = engine.connect()

In [43]:
print(pd.read_sql('SHOW TABLES', conn))

Tables_in_retail_sales
0              raw_sales


In [70]:
df = pd.read_sql('SELECT * FROM raw_sales', conn)
df.head()

,date,shop_id,item_id,item_price,item_cnt_day
0,2015-01-04,29,1469,1199.0,1.0
1,2015-01-04,28,21364,479.0,1.0
2,2015-01-04,28,21365,999.0,2.0
3,2015-01-04,28,22104,249.0,2.0
4,2015-01-04,28,22091,179.0,1.0


In [71]:
df[df.isna().any(axis=1)]

,date,shop_id,item_id,item_price,item_cnt_day


In [72]:

print(df[df.columns[df.isnull().any()]].isnull().sum().size)

0


In [73]:
duplicated = df[df.duplicated(['date','shop_id','item_id','item_price','item_cnt_day'])]
print(duplicated)
conn.close()
engine.dispose()

date  shop_id  item_id  item_price  item_cnt_day
1515 2015-01-04       29     1469      1199.0           1.0
1516 2015-01-04       28    21364       479.0           1.0
1517 2015-01-04       28    21365       999.0           2.0
1518 2015-01-04       28    22104       249.0           2.0
1519 2015-01-04       28    22091       179.0           1.0
...         ...      ...      ...         ...           ...
4540 2015-01-04       15     4240      1299.0           1.0
4541 2015-01-04       14    21922        99.0           1.0
4542 2015-01-04       15     1969      3999.0           1.0
4543 2015-01-04       14    22091       179.0           1.0
4544 2015-01-04       15     1007      1199.0           1.0

[3030 rows x 5 columns]


In [97]:
driver   = 'mysql+pymysql:'
user     = 'root'
password = '53cawentOo69'
ip       = 'localhost'
database = 'retail_sales'
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)
conn = engine.connect()

In [98]:
df.to_sql('raw_sales1', conn, if_exists='replace')


In [99]:
shop_df = df.groupby(by=['shop_id','date']).sum()
shop_df1 = df.copy()

In [100]:
df

,date,shop_id,item_id,item_price,item_cnt_day
0,2015-01-04,29,1469,1199.0,1.0
1,2015-01-04,28,21364,479.0,1.0
2,2015-01-04,28,21365,999.0,2.0
3,2015-01-04,28,22104,249.0,2.0
4,2015-01-04,28,22091,179.0,1.0
...,...,...,...,...,...
4540,2015-01-04,15,4240,1299.0,1.0
4541,2015-01-04,14,21922,99.0,1.0
4542,2015-01-04,15,1969,3999.0,1.0
4543,2015-01-04,14,22091,179.0,1.0


In [101]:
def get_sales_amount_item(row):
    return row.item_price * row.item_cnt_day

shop_df1['sales_per_item'] = shop_df1.apply(lambda row : get_sales_amount_item(row), axis=1)
shop_df1

,date,shop_id,item_id,item_price,item_cnt_day,sales_per_item
0,2015-01-04,29,1469,1199.0,1.0,1199.0
1,2015-01-04,28,21364,479.0,1.0,479.0
2,2015-01-04,28,21365,999.0,2.0,1998.0
3,2015-01-04,28,22104,249.0,2.0,498.0
4,2015-01-04,28,22091,179.0,1.0,179.0
...,...,...,...,...,...,...
4540,2015-01-04,15,4240,1299.0,1.0,1299.0
4541,2015-01-04,14,21922,99.0,1.0,99.0
4542,2015-01-04,15,1969,3999.0,1.0,3999.0
4543,2015-01-04,14,22091,179.0,1.0,179.0


In [102]:
# SALES PER SHOP
sales_shop_df = shop_df1.copy()
#sales_shop_df = sales_shop_df.groupby(by=['shop_id','date'])[['sales_per_item']].sum()
sales_shop_df = sales_shop_df.groupby(by=['shop_id']).agg({'sales_per_item':'sum'})
sales_shop_df

,sales_per_item
shop_id,
2,103746.0
3,67443.0
4,29361.0
5,33138.0
6,138678.0
7,52371.0
10,22716.0
12,295173.0
14,57450.0


In [104]:
sales_item_df = shop_df1.copy()
sales_item_df = sales_item_df.groupby(by=['item_id']).agg({'sales_per_item':'sum'})
sales_item_df

,sales_per_item
item_id,
30,507.0
31,1089.0
32,447.0
42,897.0
59,747.0
...,...
22091,1074.0
22092,537.0
22104,1494.0
